In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
# from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
# from sklearn.metrics import f1_score,accuracy_score,confusion_matrix,mean_squared_error,r2_score,mean_absolute_error
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.impute import SimpleImputer
# from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
from sklearn.decomposition import PCA
import os
import cv2
import pickle
from tqdm import tqdm

In [ ]:
def get_len(the_path):
    length=0
    categories=os.listdir(the_path)
    for category in categories:
        Path=os.path.join(the_path,category)
        length=length+len(os.listdir(Path))
    return length

In [ ]:
# load and resizing the images
def get_image_big_batch(Batch,Big_Batch_size,the_path,Batch_size,Random_state):
    Start=Big_Batch_size*(Batch//((Big_Batch_size*2)//Batch_size))
    categories=os.listdir(the_path)
    MyDataSet=[]
    for category in categories:
        label=categories.index(category)
        Path=os.path.join(the_path,category)
        if len(os.listdir(Path))>Start+Big_Batch_size:
            for pic_name in os.listdir(Path)[Start:Start+Big_Batch_size]:
                MyDataSet.append((os.path.join(Path,pic_name),label))
        else:
            for pic_name in os.listdir(Path)[Start:]:
                MyDataSet.append((os.path.join(Path,pic_name),label))
    shuffled_MyDataSet=shuffle(MyDataSet,random_state=Random_state)
    return shuffled_MyDataSet


In [ ]:
def get_batch_from_big_batch(batch,batch_size,the_big_batch,pic_size):
    Start=batch_size*(batch%(len(the_big_batch)//batch_size))
    mini_X_MyDataSet=[]
    mini_Y_MyDataSet=[]
    for path,Y in the_big_batch[Start:Start+batch_size]:
        try:
            image=cv2.imread(path)/255
            resized = cv2.resize(image, (pic_size, pic_size))
            mini_X_MyDataSet.append(resized)
            mini_Y_MyDataSet.append(Y)
        except:
            pass

    return mini_X_MyDataSet,mini_Y_MyDataSet

In [ ]:
def train_on_batch(the_path,batch_size,Big_Batch_size,n_epoch,val_num_in_batch,pic_size,seed):
    loss_history=[]
    acc_history=[]
    val_loss_history=[]
    val_acc_history=[]
    for epoch in range(n_epoch):
        # val_xds.clear()
        # val_yds.clear()
        val_xds=[]
        val_yds=[]
        for batch in tqdm(range(get_len(the_path)//batch_size)):
            if batch%(batch//((Big_Batch_size*2)//batch_size))==0:
                access='granted'
            else:
                access='not granted'
            if access=='granted':
                Big_Batch=get_image_big_batch(batch,Big_Batch_size,the_path,batch_size,Random_state=seed)
            batch_x,batch_y=get_batch_from_big_batch(batch,batch_size,Big_Batch,pic_size)
            for val_element in batch_x[-val_num_in_batch:]:
                val_xds.append(val_element)
            for val_element in batch_y[-val_num_in_batch:]:
                val_yds.append(val_element)
            loss,acc=model.train_on_batch(np.array(batch_x[:batch_size-val_num_in_batch]),np.array(batch_y[:batch_size-val_num_in_batch]))
            # val_loss,val_acc=model.test_on_batch(np.array(batch_x[-val_num_in_batch:]),np.array(batch_y[-val_num_in_batch:]))
        loss_history.append(loss)
        acc_history.append(acc)
        val_loss,val_acc=model.evaluate(np.array(val_xds),np.array(val_yds))
        del val_xds[:]
        del val_xds
        del val_yds[:]
        del val_yds
        val_loss_history.append(val_loss)
        val_acc_history.append(val_acc)
        print('Epoch: %d ,     Train Loss %.4f,    Train Acc. %.4f,    Val Loss %.4f,    Val Acc. %.4f' %
			(epoch+1, loss, acc, val_loss, val_acc))
    return {'loss_history':loss_history,'acc_history':acc_history,'val_loss_history':val_loss_history,'val_acc_history':val_acc_history}